In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [8]:
import os
cwd = os.getcwd()
project_path = (cwd, None)
while project_path[1] != "Code":
    project_path = os.path.split(project_path[0])
project_path = project_path[0]

In [9]:
data_dz = pd.read_csv(project_path + "/Datasets/data_dz.csv")
data_in = pd.read_csv(project_path + "/Datasets/data_in.csv")
data_ca = pd.read_csv(project_path + "/Datasets/data_ca.csv")

In [10]:
def generate_train_val_test_set(df, name):
    
    df = df.dropna()
    
    output = {"train": {}, "val": {}, "test": {}, "name": name}
    
    train = df.sample(frac=0.7)
    val_test = df.drop(train.index)
    
    val = val_test.sample(frac=0.7)
    test = val_test.drop(val.index)

    output["train"]["y"] = np.ravel(train[["FARE"]])
    output["val"]["y"] = np.ravel(val[["FARE"]])
    output["test"]["y"] = np.ravel(test[["FARE"]])
    
    
    output["train"]["dist"] = train[["Distance"]]
    output["val"]["dist"] = val[["Distance"]]
    output["test"]["dist"] = test[["Distance"]]

    output["train"]["simd"] = train[list(set(df.columns).difference({"ORIGIN_CODE", "DESTINATION_CODE", "Distance", "FARE"}))]
    output["val"]["simd"] = val[list(set(df.columns).difference({"ORIGIN_CODE", "DESTINATION_CODE", "Distance", "FARE"}))]
    output["test"]["simd"] = test[list(set(df.columns).difference({"ORIGIN_CODE", "DESTINATION_CODE", "Distance", "FARE"}))]

    output["train"]["full"] = train[list(set(df.columns).difference({"ORIGIN_CODE", "DESTINATION_CODE", "FARE"}))]
    output["val"]["full"] = val[list(set(df.columns).difference({"ORIGIN_CODE", "DESTINATION_CODE", "FARE"}))]
    output["test"]["full"] = test[list(set(df.columns).difference({"ORIGIN_CODE", "DESTINATION_CODE", "FARE"}))]
    
    return output.copy()

In [11]:
dataset_dz = generate_train_val_test_set(data_dz, "datazone")
dataset_in = generate_train_val_test_set(data_in, "intermediate zone")
dataset_ca = generate_train_val_test_set(data_ca, "council area")

datasets = (dataset_dz, dataset_in, dataset_ca)
subsets = ("dist", "simd", "full")

In [12]:
def fit_and_evaluate_ML_model(method, method_name, dataset, subset = "full", log_name = None, *args, **kwargs):
    model = method(*args, **kwargs)
    model.fit(dataset["train"][subset], dataset["train"]["y"])
    test_score = model.score(dataset["val"][subset], dataset["val"]["y"])

    if log_name == -1:
        log_name = method_name
    
    arg_output = "[" + ",".join(list(args) + [f"{k}->{v}" for k, v in kwargs.items()]) + "]"
    output = "|".join([method_name, dataset["name"], subset, str(test_score), arg_output])
    
    if not log_name is None:
        with open(project_path + "/Logs/" +log_name + ".txt", 'a') as log:
            log.write(output + '\n')
    else:
        print(output)
    return model

In [13]:
def check_on_all_data(method, method_name, log_name, *args, **kwargs):
    for dataset in datasets:
        for subset in subsets:
            try:
                fit_and_evaluate_ML_model(method, method_name, dataset, subset = subset, log_name = log_name, *args, **kwargs)
            except Exception as e:
                with open(project_path + "/Logs/errors.txt", 'a') as log:
                    log.write(",".join(method_name, dataset["name"], subset, str(e)))

In [14]:
from sklearn.linear_model import LinearRegression

check_on_all_data(LinearRegression, "linreg", -1)

In [9]:
from sklearn.ensemble import RandomForestRegressor

check_on_all_data(RandomForestRegressor, "rfr", -1)

In [ ]:
from sklearn.svm import SVR

kernels = ("linear", "poly", "rbf", "sigmoid")
for k in kernels:
    check_on_all_data(SVR, "svr", -1, kernel = k)